# **Data Cleaning Notebook**

**Objectives**  
Clean the data in preparation for feature engineering and modelling by:
- Dropping features with excessive missing values
- Imputing numerical and categorical missing values appropriately
- Creating a clean training and test dataset split

**Inputs**
- `outputs/datasets/collection/house_prices_records.csv`
- `outputs/datasets/collection/inherited_houses.csv`

**Outputs**
- `outputs/datasets/cleaned/TrainSetCleaned.csv`
- `outputs/datasets/cleaned/TestSetCleaned.csv`
- `outputs/datasets/cleaned/FullSetCleaned.csv`

**Additional Comments**
- Columns with >85% missing values were dropped.
- Median imputation was used for skewed numerical columns.
- Categorical imputation with 'None' was used where feature absence was implied.
- All datasets are now clean and ready for correlation study or feature engineering.


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/heritage-house-price-predictor/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/heritage-house-price-predictor'

# Load the Data

Section 1 content

In [4]:
import pandas as pd

house_prices_df = pd.read_csv("outputs/datasets/collection/house_prices_records.csv")
inherited_houses_df = pd.read_csv("outputs/datasets/collection/inherited_houses.csv")

print("House Prices Data:")
print(house_prices_df.shape)
print(house_prices_df.head())

print("\nInherited Houses Data:")
print(inherited_houses_df.shape)
print(inherited_houses_df.head())

House Prices Data:
(1460, 24)
   1stFlrSF  2ndFlrSF  BedroomAbvGr BsmtExposure  BsmtFinSF1 BsmtFinType1  \
0       856     854.0           3.0           No         706          GLQ   
1      1262       0.0           3.0           Gd         978          ALQ   
2       920     866.0           3.0           Mn         486          GLQ   
3       961       NaN           NaN           No         216          ALQ   
4      1145       NaN           4.0           Av         655          GLQ   

   BsmtUnfSF  EnclosedPorch  GarageArea GarageFinish  ...  LotFrontage  \
0        150            0.0         548          RFn  ...         65.0   
1        284            NaN         460          RFn  ...         80.0   
2        434            0.0         608          RFn  ...         68.0   
3        540            NaN         642          Unf  ...         60.0   
4        490            0.0         836          RFn  ...         84.0   

   MasVnrArea OpenPorchSF  OverallCond  OverallQual  TotalBsmt

---

# Missing Data

Define the Function

In [5]:
def EvaluateMissingData(df):
    """
    Returns a summary of columns with missing data:
    - Absolute count
    - Percentage of total rows
    - Data type
    """
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute / len(df) * 100, 2)
    df_missing_data = pd.DataFrame({
        "RowsWithMissingData": missing_data_absolute,
        "PercentageOfDataset": missing_data_percentage,
        "DataType": df.dtypes
    })
    df_missing_data = df_missing_data[df_missing_data["RowsWithMissingData"] > 0]
    df_missing_data = df_missing_data.sort_values(by="PercentageOfDataset", ascending=False)
    return df_missing_data

Apply to Our Data

In [6]:
print("House Prices Dataset - Missing Data")
missing_house_prices = EvaluateMissingData(house_prices_df)
print(missing_house_prices)

print("\nInherited Houses Dataset - Missing Data")
missing_inherited = EvaluateMissingData(inherited_houses_df)
print(missing_inherited)

House Prices Dataset - Missing Data
               RowsWithMissingData  PercentageOfDataset DataType
EnclosedPorch                 1324                90.68  float64
WoodDeckSF                    1305                89.38  float64
LotFrontage                    259                17.74  float64
GarageFinish                   235                16.10   object
BsmtFinType1                   145                 9.93   object
BedroomAbvGr                    99                 6.78  float64
2ndFlrSF                        86                 5.89  float64
GarageYrBlt                     81                 5.55  float64
BsmtExposure                    38                 2.60   object
MasVnrArea                       8                 0.55  float64

Inherited Houses Dataset - Missing Data
Empty DataFrame
Columns: [RowsWithMissingData, PercentageOfDataset, DataType]
Index: []


Drop Columns with Too Much Missing Data
* `EnclosedPorch` (90.68%)
* `WoodDeckSF` (89.38%)

In [7]:
from feature_engine.selection import DropFeatures

def drop_features(df):
    """
    Drops high-missing columns: EnclosedPorch and WoodDeckSF
    """
    imputer = DropFeatures(features_to_drop=['EnclosedPorch', 'WoodDeckSF'])
    df_removed_columns = imputer.fit_transform(df)
    return df_removed_columns

# Apply to both datasets
house_prices_df = drop_features(house_prices_df)
inherited_houses_df = drop_features(inherited_houses_df)

# Confirm shape change
print("House Prices shape:", house_prices_df.shape)
print("Inherited Houses shape:", inherited_houses_df.shape)

House Prices shape: (1460, 22)
Inherited Houses shape: (4, 21)


Impute Missing Numerical Data with Median
* `LotFrontage` (17.74% missing)
* `BedroomAbvGr` (6.78% missing)
* `2ndFlrSF` (5.89% missing)
* `GarageYrBlt` (5.55% missing)
* `MasVnrArea` (0.55% missing)

In [8]:
from feature_engine.imputation import MeanMedianImputer

# Define the variables
variables_median = ['LotFrontage', 'BedroomAbvGr', '2ndFlrSF', 'GarageYrBlt', 'MasVnrArea']

# Create and apply imputer
median_imputer = MeanMedianImputer(imputation_method='median', variables=variables_median)
df_median_imputed = median_imputer.fit_transform(house_prices_df)

# Check remaining missing values
house_prices_df = df_median_imputed
print("Rechecking missing values after assignment:")
print(house_prices_df[variables_median].isnull().sum())

Rechecking missing values after assignment:
LotFrontage     0
BedroomAbvGr    0
2ndFlrSF        0
GarageYrBlt     0
MasVnrArea      0
dtype: int64


Impute Categorical Variables
* `GarageFinish` (16.10% missing)
* `BsmtFinType1` (9.93% missing)
* `BsmtExposure` (2.60% missing)

In [9]:
from feature_engine.imputation import CategoricalImputer

variables_categorical = ['GarageFinish', 'BsmtFinType1', 'BsmtExposure']

# Create the imputer
cat_imputer = CategoricalImputer(
    imputation_method='missing',
    fill_value='None',
    variables=variables_categorical
)

# Fit and transform the data
house_prices_df = cat_imputer.fit_transform(house_prices_df)

# Recheck missing values in the categorical columns
print(house_prices_df[variables_categorical].isnull().sum())

GarageFinish    0
BsmtFinType1    0
BsmtExposure    0
dtype: int64


/home/cistudent/.local/lib/python3.12/site-packages/feature_engine/imputation/categorical.py:232: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(X[variable]):


## Final Check and Save Cleaned Data

Final Checks

In [10]:
print(" Final check: Any missing values?")
print(house_prices_df.isnull().sum().sum())  # Should be 0

print("\n Final shape of cleaned data:")
print(house_prices_df.shape)

print("\n Preview of cleaned dataset:")
house_prices_df.head()

 Final check: Any missing values?
0

 Final shape of cleaned data:
(1460, 22)

 Preview of cleaned dataset:


,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GarageYrBlt,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,548,RFn,2003.0,...,8450,65.0,196.0,61,5,7,856,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,460,RFn,1976.0,...,9600,80.0,0.0,0,8,6,1262,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,608,RFn,2001.0,...,11250,68.0,162.0,42,5,7,920,2001,2002,223500
3,961,0.0,3.0,No,216,ALQ,540,642,Unf,1998.0,...,9550,60.0,0.0,35,5,7,756,1915,1970,140000
4,1145,0.0,4.0,Av,655,GLQ,490,836,RFn,2000.0,...,14260,84.0,350.0,84,5,8,1145,2000,2000,250000


Split into Train/Test & Save

In [12]:
from sklearn.model_selection import train_test_split
import os

# Split the data
TrainSet, TestSet = train_test_split(house_prices_df, test_size=0.2, random_state=3)

# Combine for full set (useful later for inspection or correlation)
FullSet = pd.concat([TrainSet, TestSet], ignore_index=True)

# Create cleaned output folder (if it doesn't exist)
try:
    os.makedirs("outputs/datasets/cleaned", exist_ok=True)
except Exception as e:
    print(e)

# Save the files
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)
FullSet.to_csv("outputs/datasets/cleaned/FullSetCleaned.csv", index=False)

print("Cleaned datasets saved successfully!")

Cleaned datasets saved successfully!


---

NOTE

* You may add as many sections as you want, as long as they support your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.